In [ ]:
# Step 1. Install tools
!sudo apt-get install -y samtools bcftools
!conda install -c bioconda minimap2 clair3 -y

In [ ]:
# Step 2. Download HIV reference genome
!wget -O data/hiv_ref.fasta "https://www.ncbi.nlm.nih.gov/search/api/sequence/K03455.1/?report=fasta"

In [ ]:
# Step 3. Simulate ONT reads (NanoSim or ART)
!pip install nanosim-h
!nanosim-h -r data/hiv_ref.fasta -o data/hiv_reads

In [ ]:
# Step 4. Map reads
!minimap2 -a data/hiv_ref.fasta data/hiv_reads.fastq | samtools view -Sb - | samtools sort -o results/reads.bam
!samtools index results/reads.bam

In [ ]:
# Step 5. Run Clair3
!run_clair3.sh \
  --bam_fn=results/reads.bam \
  --ref_fn=data/hiv_ref.fasta \
  --threads=2 \
  --platform=ont \
  --model_path=/usr/local/lib/python3.10/dist-packages/clair3/models/ont \
  --output=results/clair3_output

In [ ]:
import pysam

resistance_db = {
    "M184V": ["3TC", "FTC"],
    "K65R": ["TDF", "ABC"],
    "K103N": ["EFV", "NVP"],
    "Y181C": ["NVP", "RPV"]
}

vcf_in = pysam.VariantFile("results/clair3_output/variants.vcf")

for rec in vcf_in.fetch():
    ref = rec.ref
    pos = rec.pos
    for alt in rec.alts:
        mutation = f"{ref}{pos}{alt}"
        if mutation in resistance_db:
            print(f"⚠️ {mutation} → Resistance to {resistance_db[mutation]}")